In [ ]:
import json
import random
import time
import threading
import paho.mqtt.client as mqtt

# Load configuration
with open("battery_machine_config.json") as f:
    machine_configs = json.load(f)

# MQTT Setup
mqtt_broker = "localhost"
mqtt_port = 1883
client = mqtt.Client()
client.connect(mqtt_broker, mqtt_port, 60)

def process_machine(machine):
    machine_id = machine["machine_name"]
    while True:
        print(f"[{machine_id}] Starting cycle")
        for step in machine["steps"]:
            value = random.gauss(step["mean"], step["variance"])
            payload = {
                "machine_id": machine_id,
                "step_name": step["step_name"],
                "value": value,
                "mean": step["mean"],
                "variance": step["variance"],
                "process_time_min": step["process_time_min"],
                "fatal_failure": step["fatal_failure"],
                "timestamp": time.strftime('%Y-%m-%d %H:%M:%S')
            }
            topic = f"factory/{machine_id}/{step['step_name']}"
            client.publish(topic, json.dumps(payload))
            print(f"[{machine_id}] Published to {topic}: {payload}")
            time.sleep(step["process_time_min"] * 60)
        print(f"[{machine_id}] Cycle complete. Restarting...\n")

# Start a thread for each machine
for machine in machine_configs:
    threading.Thread(target=process_machine, args=(machine,), daemon=True).start()

# Keep the main thread alive
while True:
    time.sleep(60)
